
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>


# Complex Transformations

Querying tabular data stored in the data intelligence platform with Spark SQL is easy, efficient, and fast.

This gets more complicated as the data structure becomes less regular, when many tables need to be used in a single query, or when the shape of data needs to be changed dramatically. This notebook introduces a number of functions present in Spark SQL to help engineers complete even the most complicated transformations.

## Learning Objectives
By the end of this lesson, you should be able to:
- Use **`.`** and **`:`** syntax to query nested data
- Parse JSON strings into structs
- Flatten and unpack arrays and structs
- Combine datasets using joins
- Reshape data using pivot tables

## REQUIRED - SELECT CLASSIC COMPUTE

Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default.

Follow these steps to select the classic compute cluster:

1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.

1. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:

  - In the drop-down, select **More**.

  - In the **Attach to an existing compute resource** pop-up, select the first drop-down. You will see a unique cluster name in that drop-down. Please select that cluster.

**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:

1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.

1. Find the triangle icon to the right of your compute cluster name and click it.

1. Wait a few minutes for the cluster to start.

1. Once the cluster is running, complete the steps above to select your cluster.

## Classroom Setup

Run the following cell to configure your working environment for this course. It will also set your default catalog to **dbacademy** and the schema to your specific schema name shown below using the `USE` statements.
<br></br>


```
USE CATALOG dbacademy;
USE SCHEMA dbacademy.<your unique schema name>;
```

**NOTE:** The `DA` object is only used in Databricks Academy courses and is not available outside of these courses. It will dynamically reference the information needed to run the course.

In [0]:
USE ${catalog_name}.${schema_name}


## Data Overview

The **`events_raw`** table was registered against data representing a Kafka payload. In most cases, Kafka data will be binary-encoded JSON values. 

Let's cast the **`key`** and **`value`** as strings to view these values in a human-readable format.

In [0]:
CREATE OR REPLACE TEMP VIEW events_strings AS 
SELECT 
  CAST(key AS string) AS key,  string(value) 
FROM events_raw;



SELECT * 
FROM events_strings LIMIT 10;

As we can see from the results above, the data consists of a unique key and a JSON string of event data.

## Convert STRING to VARIANT

In [0]:
SELECT * from store_data

In [0]:
CREATE OR REPLACE TABLE events_variant
AS 
SELECT key as variant_key, parse_json(value) as variant_value  
FROM events_strings

## Variant null rules
Variants can contain two kinds of nulls:

- **SQL NULL:** SQL `NULL`s indicate that the value is missing. These are the same `NULLs` as when dealing with structured data.
- **Variant NULL:** Variant `NULLs` indicate that the variant explicitly contains a `NULL` value. These are not the same as SQL `NULLs`, because the `NULL` value is stored in the data.

Use the `is_variant_null` function to determine if the variant value is a variant `NULL`.

In [0]:
SELECT
  is_variant_null(parse_json(NULL)) AS sql_null,
  is_variant_null(parse_json('null')) AS variant_null,
  is_variant_null(parse_json('{ "field_a": null }'):field_a) AS variant_null_value,
  is_variant_null(parse_json('{ "field_a": null }'):missing) AS missing_sql_value_null


## Manipulate Variant Data

### Work with Nested Data
Spark SQL has built-in functionality to directly interact with nested data stored as JSON strings or struct types.
- Use **`:`** syntax in queries to access subfields in JSON strings
- Use **`.`** syntax in queries to access subfields in struct types

Let's step into the **`value`** column and grab one row of data with an **`event_name`** of "finalize."

In [0]:
SELECT * 
FROM events_strings 
WHERE value:event_name = "finalize" 
ORDER BY key 
LIMIT 1;

In [0]:
WITH base AS 
(
  SELECT * , CAST(variant_value:event_name AS STRING) AS event_name
FROM events_variant 
)
SELECT * EXCEPT (event_name) FROM base WHERE event_name = "finalize" 
ORDER BY variant_key 
LIMIT 1;

In [0]:
SELECT * , variant_value:event_name AS event_name
FROM events_variant 


In [0]:
SELECT 
 variant_value:items[0].item_id AS item_id_0
 ,variant_value:['ecommerce']:purchase_revenue_in_usd USD
 FROM events_variant

## Schema inference with Variant data

In [0]:
SELECT  schema_of_variant(parse_json(value)), count(*) AS schema FROM events_strings
GROUP BY schema_of_variant(parse_json(value))

In [0]:
-- SELECT schema_of_variant_agg(a) FROM VALUES(parse_json('{"foo": "bar"}')) AS data(a);
SELECT schema_of_variant_agg(a) FROM (SELECT parse_json(value) AS a FROM events_strings) AS Data

In [0]:
SELECT CAST(variant_value AS STRUCT<device: STRING, ecommerce: STRUCT<purchase_revenue_in_usd: DECIMAL(5,1), total_item_quantity: BIGINT, unique_items: BIGINT>, event_name: STRING, event_previous_timestamp: BIGINT, event_timestamp: BIGINT, geo: STRUCT<city: STRING, state: STRING>, items: ARRAY<STRUCT<coupon: STRING, item_id: STRING, item_name: STRING, item_revenue_in_usd: DECIMAL(18,14), price_in_usd: DECIMAL(4,0), quantity: BIGINT>>, traffic_source: STRING, user_first_touch_timestamp: BIGINT, user_id: STRING>) from events_variant

In [0]:
SELECT try_variant_get(parse_json('{"key": 123, "data": [4, {"a": "hello"}, "str"]}'), '$.data', 'array<int>')
  null

In [0]:
-- Index elements
SELECT 
 variant_value:items[0] items_0
,variant_value:items[1] items_1 
,variant_value:items[2] items_2 
 FROM events_variant

Let's use the **JSON** string example above to derive the schema, then parse the entire **JSON** column into **STRUCT** types.
- **`schema_of_json()`** returns the schema derived from an example **JSON** string.
- **`from_json()`** parses a column containing a **JSON** string into a **STRUCT** type using the specified schema.

After we unpack the **JSON** string to a **STRUCT** type, let's unpack and flatten all **STRUCT** fields into columns.

**`*`** unpacking can be used to flatten a **STRUCT**; **`col_name.*`** pulls out the subfields of **`col_name`** into their own columns.

In [0]:
SELECT schema_of_variant(parse_json('{"device":"Linux","ecommerce":{"purchase_revenue_in_usd":1075.5,"total_item_quantity":1,"unique_items":1},"event_name":"finalize","event_previous_timestamp":1593879231210816,"event_timestamp":1593879335779563,"geo":{"city":"Houston","state":"TX"},"items":[{"coupon":"NEWBED10","item_id":"M_STAN_K","item_name":"Standard King Mattress","item_revenue_in_usd":1075.5,"price_in_usd":1195.0,"quantity":1}],"traffic_source":"email","user_first_touch_timestamp":1593454417513109,"user_id":"UA000000106116176"}')) AS schema;

In [0]:
--CREATE OR REPLACE TEMP VIEW parsed_events AS 

  SELECT cast(variant_value as MAP<STRING, STRING>) AS json 
  FROM events_variant;

-- SELECT * 
-- FROM parsed_events;

In [0]:
SELECT
variant_value:items 
FROM  events_variant

In [0]:
CREATE OR REPLACE TEMP VIEW parsed_events_variant AS 
SELECT json.*, items_variant , variant_value FROM
(  
  SELECT variant_value, variant_value:items as items_variant, cast(variant_value as STRUCT<device: STRING, ecommerce: STRUCT<purchase_revenue_in_usd: DECIMAL(5,1), total_item_quantity: BIGINT, unique_items: BIGINT>, event_name: STRING, event_previous_timestamp: BIGINT, event_timestamp: BIGINT, geo: STRUCT<city: STRING, state: STRING>, items: ARRAY<STRUCT<coupon: STRING, item_id: STRING, item_name: STRING, item_revenue_in_usd: DECIMAL(5,1), price_in_usd: DECIMAL(4,0), quantity: BIGINT>>, traffic_source: STRING, user_first_touch_timestamp: BIGINT, user_id: STRING>) AS json 
  FROM events_variant
);
SELECT * 
FROM parsed_events_variant;

### Manipulate Arrays

Spark SQL has a number of functions for manipulating array data, including the following:
- **`explode()`** separates the elements of an array into multiple rows; this creates a new row for each element.
- **`size()`** provides a count for the number of elements in an array for each row.

The code below explodes the **`items`** field (an array of structs) into multiple rows and shows events containing arrays with 3 or more items.

In [0]:

SELECT *
from parsed_events_variant,
LATERAL variant_explode_outer(items_variant)


In [0]:
select size(cast(items_variant as array<variant>)) from parsed_events_variant

In [0]:
CREATE OR REPLACE TEMP VIEW exploded_events AS
SELECT  *, 
  explode_outer(items) AS item
FROM parsed_events;

In [0]:
-- Find users with more than 2 items in their cart
SELECT * 
FROM exploded_events_variant 
WHERE size(items) > 2;

In [0]:
-- Confirm data type for new exploded 'item' column
DESCRIBE exploded_events;

## Nesting Functions
We may want to see a list of all events associated with each **`user_id`** and we can collect all items that have been in a user's cart at any time for any event. Let's walk through how we can accomplish this.
### Step 1
We use **`collect_set()`** to gather ("collect") all unique values in a group, including arrays. We use it here to collect all unique **`item_id`**'s in our **`items`** array of structs.


In [0]:
WITH BASE AS 
(SELECT * except (exploded_events_variant.value)
from exploded_events_variant,
LATERAL variant_explode_outer(variant_value:items))
,SUBSET AS
(SELECT user_id, BASE.value as item from BASE)
SELECT 
  user_id,
  Collect_set(variant_get(item,'$.item_id')) AS cart_history
FROM SUBSET
GROUP BY user_id
ORDER BY user_id;

In [0]:
WITH BASE AS 
(SELECT * except (exploded_events_variant.value)
from exploded_events_variant,
LATERAL variant_explode_outer(variant_value:items))
,SUBSET AS
(SELECT user_id, BASE.value as item from BASE)
SELECT 
  user_id,
  array_distinct(
    flatten(
      array(
        Collect_set(
          CAST(
            variant_get(item,'$.item_id')
             AS STRING
             )
      )
    )
  )
 ) AS cart_history
FROM SUBSET
GROUP BY user_id
ORDER BY user_id;

In this case, our result for the list of **`item_id`**'s is an array nested in another array, so let's use **`flatten()`** to pull all items into a single array.


Because there were multiple sets of items involved, there are duplicate values in our array. We use **`array-distinct()`** to remove these duplicates.

In [0]:
SELECT 
  user_id,
  collect_set(event_name) AS event_history,
  array_distinct(flatten(collect_set(items.item_id))) AS cart_history
FROM exploded_events
GROUP BY user_id
ORDER BY user_id;


&copy; 2025 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the 
<a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use">Terms of Use</a> | 
<a href="https://help.databricks.com/">Support</a>